In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy import interpolate
from scipy.optimize import approx_fprime

plt.rcParams.update({'font.size': 14})
%config InlineBackend.figure_format = 'retina'

%matplotlib notebook
from matplotlib.animation import FuncAnimation

In [ ]:
%matplotlib inline
# approx dimensions http://www.leelikesbikes.com/build-your-own-pump-track.html

# lenghts in meters

n = 1000
end = 2.5
freq = 2 * np.pi / end
l = np.linspace(0, end, n)
phase_shift = 0
k = 0.5
# phi = k * np.cos(l + phase_shift)
cos_a = -k * np.sin(freq * l + phase_shift)
d_cos_a_dl = -k * freq * np.cos(freq * l + phase_shift)
sin_a = (1 - cos_a**2)**0.5

dl = l[1] - l[0]
y_ground = np.cumsum(cos_a * dl)
x_ground = np.cumsum(sin_a * dl)
print(f"height = {y_ground.max() - y_ground.min()}, lenght = {x_ground.max() - x_ground.min()}")

plt.figure(figsize=(10,4))
plt.plot(x_ground, y_ground)

xlim = (x_ground[0] - 1, x_ground[-1] + 1)
ylim = (-1, 0.5)
plt.grid()
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim(xlim)
plt.ylim(ylim)

cos_a = interpolate.interp1d(l, cos_a) 
sin_a = interpolate.interp1d(l, sin_a) 
d_cos_a_dl = interpolate.interp1d(l, d_cos_a_dl) 
x_ground = interpolate.interp1d(l, x_ground) 
y_ground = interpolate.interp1d(l, y_ground) 
# phi = interpolate.interp1d(l, phi) 


In [ ]:
# x = np.arange(0, 10)
# y = np.exp(-x/3.0)
# f = interpolate.interp1d(x, y)

r = lambda l : 0
g = 9.8
h0 = 1
vl0 = 8
al, vl, xl = 0, vl0, [phase_shift]
ah, vh, xh = 0, -vl * cos_a(xl[-1]), [h0]

dt = 0.003
i = 0
u = []

while True:
    cos_a_i = cos_a(xl[-1])
    ri = r(xl[-1])
    ah = -(vh + 1.5 * vl * cos_a_i) / dt
    u.append(ah)
#     ah = u[i]
#     ah = -(vh + vl * cos_a_i) / dt
#     ah = 0
#     ah = -al * cos_a_i - (vl)**2 * phi(xl[-1])
#     ah = 
#     al = -(g + ri) * cos_a_i / (1 - cos_a_i**2)
    al = -(g + ah) * cos_a_i
    
    vl += al * dt
    xl_next = xl[-1] + vl * dt + al * dt**2 / 2
    if xl_next > l[-1]:
        break
    xl.append(xl_next)
    
#     ah = ri - al*cos_a_i
    vh += ah * dt
    xh.append(xh[-1] + vh * dt + ah * dt**2 / 2)
    
    i += 1
    
# plt.plot(l, (np.cos(l/0.935)-1)/2)
u = np.array(u)
print(vl, vl0 + dt*(-(g+u)*cos_a(np.array(xl))).sum())

def f(u, xl):
    return (-(g+u)*cos_a(np.array(xl))).sum()

# def df(u):
    

assert(abs(vl - vl0 - dt * f(u, xl)) < 1e-4 )

In [ ]:
u.min() / g, u.max() / g

In [ ]:
x_ground(xl[-1])

In [ ]:
class Env:
    def __init__(self, al0=0, ah0=0, vl0=5 / 18 * 5, vh0=None, xl0=0, xh0=0):
        self.al = al0
        self.ah = ah0
        self.vl0 = vl0
        self.vh0 = -vl0 * cos_a(xl0) if vh0 is None else vh0
        assert self.vh0 == 0.0
#         print(self.vh0)
        self.vl = [vl0]
        self.xl = [xl0]  # x_l(t)
        self.xh = [xh0]
    
    def ride(self, u):
        t = 0
        vl, vh = self.vl0, self.vh0
        T = len(u)
        
        for t in range(T):
            cos_a_i = cos_a(self.xl[-1])
            ah = u[t]
            al = -(g + ah) * cos_a_i
            vl += al * dt
            xl_next = self.xl[-1] + vl * dt + al * dt ** 2 / 2
            if xl_next > l[-1]:
                break
            self.xl.append(xl_next)

        #     ah = ri - al*cos_a_i
            vh += ah * dt
            self.xh.append(self.xh[-1] + vh * dt) #+ ah * dt ** 2 / 2)
            self.vl.append(vl)
        return self.xl, self.xh
            
    def _dl_next(self):
        self.dl.append(self.dl[-1] + self.dv[-1] * dt + self.da[-1] * dt ** 2 / 2)
        
    def grad(self, u):
        df = []
        xl = self.xl[:-1]  # len(self.xl) = len(u) + 1, because of last point. grad_f dont depend on self.xl[-1]
        dcos_a = d_cos_a_dl(xl)
        dim = len(u)
        u = u[:len(xl)]  # when xl(T) > l[-1] (if len(xl) < len(u)), objective dont depend on u[len(xl): ]
        T = len(u)
        for t in range(T):
            self.dv = [0]  # d vl /du_t
            self.dl = [0]  # d xl /du_t
            self.da = [- cos_a(xl[t])] 
            for s in range(1, T-t):  # shift
                self._dl_next()
                self.dv.append(self.dv[-1] + self.da[-1] * dt)
#                 self.da.append((g + u[t+s]) * sin_a(xl[t+s]) * self.dl[-1])
                self.da.append(-(g + u[t+s]) * dcos_a[t+s] * self.dl[-1])
#                 self._dl_next()
#             df.append(-cos_a(xl[t]) + ((g + u[t+1:]) * sin_a(xl[t+1:]) * self.dl[1:]).sum())
#             print(t, dcos_a.shape, u.shape)
            df.append(-cos_a(xl[t]) + (-(g + u[t+1:]) * dcos_a[t+1:] * self.dl[1:]).sum())
        if len(df) < dim:
            df += [0] * (dim - len(df))
        return np.array(df)
        
        
    
    

In [ ]:
def fu(u):
    env = Env()
    env.ride(u)
    return env.vl[-1] / dt

def dfu(u):
    env = Env()
    env.ride(u)
#     print(len(env.xl))
    df = env.grad(u)
    return df

# %timeit dfu(u)
# %timeit approx_fprime(u, fu)
    

In [ ]:
T = len(u)

env = Env()
u_ = u + 0.01 * np.random.random(len(u))
env.ride(u_)
df = env.grad(u_)

xl, xh = env.xl, env.xh

In [ ]:
u.shape

In [ ]:
%matplotlib inline 

u_ = np.zeros(len(u)) + 3 * np.random.rand(len(u))
# u_ = np.linspace(20, 0, len(u)) - 10 * np.random.rand(len(u))
# u_ = u

b = dfu(u_)
a = approx_fprime(u_, fu, epsilon=1e-8)

fig = plt.figure()
# plt.plot(u, label='u')
plt.plot(100*a, label='scipy')
plt.plot(100*b, label='mine')
# plt.plot(100*-cos_a(np.array(env.xl)), label='df_u xl fixed')
# plt.plot(u-100*df, label='u-100df')
plt.legend()
plt.show()

In [ ]:
def plot_ride(xl, xh, i):
    plt.plot(x_ground(xl), y_ground(xl) + xh, '-', label=str(i))

In [ ]:
%%time
%matplotlib inline 

# ui = u.copy()
ui = - 10 * np.resize(y_ground(xl), 500)  # starting point
# np.zeros(400)
h = 2 
flog, steplog = [], []
umin = -g 
umax = g * 2
for i in range(100):
    env = Env()
    xl, xh = env.ride(ui)
#     plot_ride(xl, xh, i)
    df = env.grad(ui)
    ui_prev = ui.copy()
    ui += h * df
    
    # ui -= ui.sum() / ui.size  # projection
    
    # project (TODO: analytic projection on intersection instead of alternating)
    proj_N = 25
    for j in range(proj_N):
        # project on box
        ui_pre_proj = ui.copy()
        ui = np.clip(ui, umin, umax)
        box_norm = np.linalg.norm(ui_pre_proj - ui, 1)
        if j > 0 and box_norm < 1e-5 * ui.size :
            break
        elif j == proj_N - 1:
            print("proj fail")
        
#         print(f"{j}, box constr l1 norm: {}")
        
        # project on x_h[-1] = 0  # why????
#         T = len(env.xh) - 1  # dont work since xh was calculated before projection on box
#         xh = np.sum(np.cumsum(ui[:T] * dt) * dt) 
#         print(i, j, "before proj", env.xh[-1])
#         continue # skip proj on x_h[-1] = 0
        env_ = Env()
        env_.ride(ui)
#         print(i, j, "before proj", env_.xh[-1], "\n")
        xh = env_.xh[-1]
        T = len(env_.xh) - 1
        du = 1/ dt**2 * xh * 6 / (T * (T + 1) * (2 * T + 1)) * (np.arange(T) - T)
        du.resize(len(ui))
#         print(du)
        ui += du
        xh = np.sum(np.cumsum(ui * dt) * dt)
#         with open("tmp.dump", "w") as fp:
#         np.savetxt("cum.dump", np.cumsum(np.cumsum(ui * dt) * dt))
#         print(i, j, "after proj", xh)
#         env_ = Env()
#         env_.ride(ui)
#         np.savetxt("env.dump", env_.xh)
#         print(i, j, "after proj", env_.xh[-1], "\n")
#         assert j < 0
#     print(fu(ui))
    flog.append(fu(ui))
    steplog.append(np.linalg.norm(ui - ui_prev))


In [ ]:
%matplotlib inline
fig, ax1 = plt.subplots()
ax1.plot(x_ground(l), y_ground(l), 'b')
h0 = 0.3
plot_ride(env.xl, np.array(env.xh) + h0, "traj")
# ax1.plot(x_ground(xl), cos_a(xl), label="cos")
# ax1.plot(x_ground(xl), ui, label="u")
ax1.plot(x_ground(xl), np.array(env.xh), label="xh")

plt.legend()
ax2 = ax1.twinx()
c = "purple"
ax2.set_ylabel("speed", color=c)
ax2.plot(x_ground(env.xl), np.array(env.vl), label="speed", color=c)

# ax3 = ax3.twinx()
plt.legend()
# plt.show()
# plt.plot(flog)
    

In [ ]:
env = Env()
xl, xh = env.ride(ui)
plt.plot(x_ground(xl), xh, label="xh")
# plt.plot(x_ground(l), y_ground(l), 'b')
# plot_ride(xl, xh, "traj")
plt.legend()
plt.show()
plt.plot(x_ground(xl), ui[:len(xl)], label="u")
plt.title("u")
plt.legend()

In [ ]:
plt.plot(steplog)
plt.yscale("log")
_ = plt.title("$||u_k - u_{k-1}||_2$")

In [ ]:
plt.plot(np.array(flog) * dt)
_ = plt.title("final speed ($u_k$), or f * dt")

In [ ]:
%matplotlib notebook

# Initialize the movie
fig = plt.figure()
# n = 1000
# x = np.linspace(0, 6*np.pi, n)
# y = np.sin(x)

# plot the sine wave line
ground, = plt.plot(x_ground(l), y_ground(l), 'b')
h0 = 0.5 - np.array(env.xh).min()
plt.plot(x_ground(env.xl), y_ground(env.xl) + env.xh + h0, 'g-')
line, = plt.plot([], [])
lower_circle, = plt.plot([], [], 'ro', markersize = 10)
upper_circle, = plt.plot([], [], 'ro', markersize = 10)

plt.xlim(xlim)
# plt.ylim(ylim+1)
# plt.xlabel('x')
# plt.ylabel('sin(x)')

# Update the frames for the movie
def update(i):
    xl_i, xh_i = env.xl[i], env.xh[i]
    x, y = x_ground(xl_i), y_ground(xl_i)
    lower_circle.set_data(x, y)
    upper_circle.set_data(x, y + xh_i + h0)
    line.set_data([x, x], [y, y+ xh_i + h0])
        

ani = FuncAnimation(fig, update, frames=np.arange(len(xl)),
                    interval=5)
plt.show()